In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re

In [2]:
# Call dataset
acm = pd.read_excel("./paper-links/PaperLinks_ACM.xlsx")

In [ ]:
# Get the paper metadata
list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []
index = 1
for paper_url in acm["Paper Links"]:
    # Get page
    page_bs = requests.get(paper_url)
    parsed_bs = bs(page_bs.text,'html.parser')    
    # Get Year
    year = parsed_bs.find(class_="epub-section__date")
    year = re.search('<span class="epub-section__date">(.*)</span>', str(year)).group(1).split(" ", 1)[1].strip()
    # Get Title
    title = re.search('<title>(.*)</title>', str(parsed_bs)).group(1).strip()
    # Get Conf/Journal Name
    conf_jour_name = parsed_bs.find(class_="epub-section__title")
    conf_jour_name = re.search('<span class="epub-section__title">(.*)</span>', str(conf_jour_name)).group(1).strip()
    # Get Authors
    authors_list = parsed_bs.find_all(class_="author-name")
    for i in range(0,len(authors_list)):
        if i == 0:
            authors = re.search('title="(.*)"><span class="loa__author-info">', str(authors_list[i])).group(1).strip()
        else:
            authors = authors+", "+re.search('title="(.*)"><span class="loa__author-info">', str(authors_list[i])).group(1).strip()
        if i == len(authors_list)-1:
            authors = authors.strip()
            break
    # Get Keywords
    keywords = "no_keyword"
    # Get Abstract
    abstract = parsed_bs.find(class_="abstractSection abstractInFull") # cleaning needed
    abstract = re.search('<p>(.*)</p>', str(abstract)).group(1).strip()
    abstract = re.sub('<.*?>', '', abstract)
    # Append to the list
    list_year.append(year)
    list_title.append(title)
    list_conf_jour.append(conf_jour_name)
    list_authors.append(authors)
    list_keywords.append(keywords)
    list_abstract.append(abstract)
    print(index)
    index += 1

In [4]:
# Concatenate list as new column
acm["Year"], acm["Title"], acm["Conf/Journal Name"] = list_year, list_title, list_conf_jour
acm["Authors"], acm["Keywords"], acm["Abstract"] = list_authors, list_keywords, list_abstract

In [5]:
acm.to_excel("./paper-metadata/PaperMetadata_Unclean_ACM.xlsx", index = False)